# Homework - Neural networks - Part C (20 points)
# === CCM LAB EDITION ===
## A neural network model of semantic cognition


<img src="semcog_net.jpeg" style="width: 450px;"/>


In [ ]:
# Import libraries
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid, relu
from scipy.cluster.hierarchy import dendrogram, linkage

Let's first load in the names of all the items, attributes, and relations into Python lists.

In [ ]:
with open('data/sem_items.txt','r') as fid:
    names_items = np.array([l.strip() for l in fid.readlines()])
with open('data/sem_relations.txt','r') as fid:
    names_relations = np.array([l.strip() for l in fid.readlines()])
with open('data/sem_attributes.txt','r') as fid:
    names_attributes = np.array([l.strip() for l in fid.readlines()])
        
# number of objects / relations / attributes
nobj = len(names_items)
nrel = len(names_relations)
nattributes = len(names_attributes)

print('List of items:')
print(names_items)
print(nobj)

print("List of relations:")
print(names_relations)
print(nrel)

print("List of attributes:")
print(names_attributes)
print(nattributes)

Next, let's load in the data matrix from a text file too. The matrix `D` has a row for each training pattern. It is split into a matrix of input patterns `input_pats` (item and relation) and their corresponding output patterns `output_pats` (attributes). The are `N` patterns total in the set.

For each input pattern, the first 8 elements indicate which item is being presented, and the next 4 indicate which relation is being queried. Each element of the output pattern corresponds to a different attribute. All patterns use 1-hot encoding.

In [ ]:
# Next, let's load in the data matrix from a text file too.
D = np.loadtxt('data/sem_data.txt')
# The matrix D has a row for each training pattern.

# It is split into a matrix of input patterns input_pats (item and relation) 
input_pats = D[:,:nobj+nrel]
input_pats = torch.tensor(input_pats,dtype=torch.float)

# and their corresponding output patterns output_pats (attributes).
output_pats = D[:,nobj+nrel:]
output_pats = torch.tensor(output_pats,dtype=torch.float)

# The are N patterns total in the set.
N = input_pats.shape[0] # number of training patterns

input_v = input_pats[0,:].numpy().astype('bool')
output_v = output_pats[0,:].numpy().astype('bool')

print('Example input pattern:')
print(input_v.astype('int'))

print('Example output pattern:')
print(output_v.astype('int'))

print("")

print("Which encodes...")
print('Item ',end='')
print(names_items[input_v[:8]])

print('Relation ',end='')
print(names_relations[input_v[8:]])

print('Attributes ',end='')
print(names_attributes[output_v])

# ========= CCM LAB EDITION ============


<img src="ugly.jpg" style="width: 350px;"/>
"Auto-encoder"


In [ ]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size):
        super(Net, self).__init__()
        self.i2h = nn.Linear(8+4,15)
        self.h2o = nn.Linear(15,8+4)
        # add code

    def forward(self, x):
        x = x.view(-1,nobj+nrel) # reshape as size [B x (nobj+nrel) Tensor] if B=1
        hidden = self.i2h(x)
        hidden = relu(hidden)
        output = self.h2o(hidden)
        output = sigmoid(output)
        return output, hidden


We provide a completed function `train` for stochastic gradient descent. The network makes online (rather than batch) updates, adjusting its weights after the presentation of each input pattern.

In [ ]:
def train(mynet,epoch_count,nepochs_additional=5000):
    #  mynet : Net class object
    #  epoch_count : (scalar) how many epochs have been completed so far
    #  nepochs_additional : (scalar) how many more epochs we want to run
    mynet.train()
    for e in range(nepochs_additional): # for each epoch
        error_epoch = 0.
        perm = np.random.permutation(N)
        for p in perm: # iterate through input patterns in random order
            mynet.zero_grad() # reset gradient
            output, hidden = mynet(input_pats[p,:]) # forward pass
            target = input_pats[p,:] 
            loss = criterion(output, target) # compute loss
            loss.backward() # compute gradient 
            optimizer.step() # update network parameters
            error_epoch += loss.item()
        error_epoch = error_epoch / float(N)        
        if e % 50 == 0:
            print('epoch ' + str(epoch_count+e) + ' loss ' + str(round(error_epoch,3)))
    return epoch_count + nepochs_additional

In [ ]:
learning_rate = 0.1
criterion = nn.MSELoss() # mean squared error loss function
mynet = Net(rep_size=8,hidden_size=15)
optimizer = torch.optim.SGD(mynet.parameters(), lr=learning_rate) # stochastic gradient descent

epoch_count = 0
epoch_count = train(mynet,epoch_count,nepochs_additional=1000)


In [ ]:
input_v = input_pats[0,:].numpy().astype('bool')

print('Example input pattern:')
print(input_v.astype('int'))

print('Item ',end='')
print(names_items[input_v[:8]])

print('Relation ',end='')
print(names_relations[input_v[8:]])

In [ ]:
output, hidden = mynet(input_pats[0,:]) # forward pass

In [ ]:
output

if time, talk about concatenation 

In [ ]:
# concatenated = torch.cat((layer1,layer2),1)